In [14]:
import json
import pandas as pd

In [15]:
with open('C:/CODE/SIC_BigData_Real-Estate-Price-Prediction/data/NhaO/NhaO_raw-dataset.json','r',encoding='utf-8') as file:
    data = json.load(file)
df1 = pd.json_normalize(data)
df1.drop_duplicates(inplace=True)

In [16]:
df1.drop(columns=['DiaChi','City','District','PhapLy','HuongNha','NoiThat'])

,DienTich,MucGia,Lat,Long,MatTien,DuongVao,SoTang,SoPhongNgu,SoToilet,HuongBanCong
0,100 m²,"8,5 tỷ",10.860251,106.756762,NaN,NaN,NaN,NaN,NaN,NaN
1,440 m²,58 tỷ,10.845933,106.795991,16 m,NaN,NaN,NaN,NaN,NaN
2,250 m²,"15,5 tỷ",10.907015,106.758006,8 m,16 m,NaN,NaN,NaN,NaN
3,31 m²,"3,4 tỷ",21.087970,105.784522,NaN,NaN,NaN,NaN,NaN,NaN
4,120 m²,"14,8 tỷ",10.815044,106.672941,8 m,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
91164,105 m²,"21,5 tỷ",20.988178,105.793152,"7,5 m",NaN,7 tầng,NaN,NaN,NaN
91165,40 m²,"3,3 tỷ",10.754977,106.629505,"3,35 m",5 m,2 tầng,3 phòng,1 phòng,NaN
91168,"39,5 m²","6,43 tỷ",10.745062,106.675805,"3,5 m",3 m,3 tầng,4 phòng,2 phòng,NaN
91171,177 m²,"99,8 tỷ",21.026387,105.832239,"6,5 m",20 m,10 tầng,NaN,NaN,NaN


In [17]:
#xử lý diện tích
df1["DienTich"] = df1["DienTich"].str.replace(' m²','').str.replace('.', '').str.replace(',', '.').str.strip()

In [18]:
#Hàm tính toán giá nhà
def clean_price(row):
    price = row['MucGia']
    area = row['DienTich']

    # Đổi kiểu cột DienTich thành float
    area = float(area)

    # Đảm bảo price là chuỗi
    if isinstance(price, (float, int)):
        return price

    price = str(price)
    
    #chuyển đổi đơn vị giá -> đồng
    if 'triệu/m²' in price:
        price_value = float(price.replace(' triệu/m²', '').replace(',', '.').strip()) * 1000000
        return price_value * area
    elif 'thỏa thuận' in price:
        return price
    elif 'nghìn/m²' in price:
        price_value = float(price.replace(' nghìn/m²', '').replace(',', '.').strip()) * 1000
        return price_value * area
    elif 'tỷ/m²' in price:
        price_value = float(price.replace(' tỷ/m²', '').replace(',', '.').strip()) * 1000000000
        return price_value * area
    elif 'triệu' in price:
        price_value = float(price.replace(' triệu', '').replace(',', '.').strip()) * 1000000
        return price_value
    elif 'tỷ' in price:
        price_value = float(price.replace(' tỷ', '').replace(',', '.').strip()) * 1000000000
        return price_value
    elif 'nghìn' in price:
        price_value = float(price.replace(' nghìn', '').replace(',', '.').strip()) * 1000
        return price_value

    return price

# Áp dụng hàm vào DataFrame
df1['MucGia'] = df1.apply(clean_price, axis=1)
print(df1['MucGia'])

0         8500000000.0
1        58000000000.0
2        15500000000.0
3         3400000000.0
4        14800000000.0
             ...      
91164    21500000000.0
91165     3300000000.0
91168     6430000000.0
91171    99800000000.0
91174     9900000000.0
Name: MucGia, Length: 45925, dtype: object


In [19]:
#Gán giá trị trung vị cho 'thỏa thuận'
# Tính giá trị trung vị
median_price = df1[df1['MucGia'] != 'Thỏa thuận']['MucGia'].astype(float).median()

# Thay thế giá trị "thỏa thuận"
df1['MucGia'] = df1['MucGia'].replace('Thỏa thuận', median_price)

C:\Users\quanp\AppData\Local\Temp\ipykernel_8552\1240423143.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1['MucGia'] = df1['MucGia'].replace('Thỏa thuận', median_price)


In [20]:
#xử lý diện tích mặt tiền
df1["MatTien"] = df1["MatTien"].str.replace(' m','').str.replace(',','.').str.strip()

#xử lý diện tích đường vào
df1["DuongVao"] = df1["DuongVao"].str.replace(' m','').str.replace(',','.').str.strip()

#xử lý diện tích số phòng ngủ
df1["SoPhongNgu"] = df1["SoPhongNgu"].str.replace(' phòng','').str.strip()

#xử lý số phòng Toilet
df1["SoToilet"] = df1["SoToilet"].str.replace(' phòng','').str.strip()

#xử lý số tầng
df1["SoTang"] = df1["SoTang"].str.replace(' tầng','').str.strip()